In [9]:
from app.utils import create_pad_sequences, clean_text, load_model, load_tokenizer, BODY_COL_NAME
from typing import Dict, Any
from deep_translator import GoogleTranslator
import pandas as pd

ROOT_DIR = '../../'

In [10]:
model_by_name: Dict[str, Dict[str, Any]] = {
    'phishing': {
        'threshold': 0.985,
    },
    'fraud': {
        'threshold': 0.90,
    },
    'spam': {
        'threshold': 0.98,
    },
}

for name in model_by_name.keys():
    model_by_name[name]['model'] = load_model(f'{ROOT_DIR}models/{name}')
    model_by_name[name]['tokenizer'] = load_tokenizer(f'{ROOT_DIR}tokenizers/{name}')

Модель загружена!


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 10, 128)        │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,379,075 (5.26 MB)

 Trainable params: 1,379,073 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

EOFError: Ran out of input

In [11]:
def predict(message: str):
    message = clean_text(message)
    df = pd.DataFrame([message], columns=[BODY_COL_NAME])

    for name, data in model_by_name.items():
        tokenizer = data['tokenizer']
        model = data['model']
        message_pad = create_pad_sequences(tokenizer, df[BODY_COL_NAME])
        prediction = model.predict(message_pad)
        prediction = prediction[0][0]
        print(name, prediction, f'Trigger: {prediction > data["threshold"]}')

In [13]:
import sys

message = input("Enter eMAIL message: ")

if not message:
    sys.exit(1)

translator = GoogleTranslator(source='auto', target='en')
message = translator.translate(message)
print(message)
predict(message)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): translate.google.com:443
DEBUG:urllib3.connectionpool:https://translate.google.com:443 "GET /m?tl=en&sl=auto&q=test HTTP/11" 200 None


test


KeyError: 'tokenizer'